# MNIST

In [6]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [1]:
import sys
sys.path.append("../")
#sys.path.append("../../extra-package")

In [2]:
import gurobipy
#import gurobi901 as gurobipy
from json import dumps, loads
from time import time

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression as skLogisticRegression
from sklearn.metrics import (classification_report, f1_score, precision_score, recall_score)
from tqdm import tnrange, trange
import tensorflow as tf

from mlsql import InfluenceRanker, SelfLossInfluenceRanker, AutoFixer, ModelManager, LossRanker, TiresiasRanker, multi_ambiguity_count
from mlsql.models import SimpleCNN, LogReg
from mlsql.utils import setdiff1d
from processors.adultNoCorr import AdultNoCorrProcessor

from itertools import groupby
from functools import partial

import logging
logging.getLogger("tensorflow").setLevel(logging.CRITICAL)

$-\nabla_\theta \mathcal{L}(z_{test}, \hat \theta) \mathcal{H}^{-1}_{\hat{\theta}} \nabla_{\theta} \mathcal{L}(\mathbf{z}, \hat \theta)$

In [3]:
import time
import altair as alt
alt.data_transformers.disable_max_rows()

@tf.function
def rank_fix(ranker, fixer, n):
    rank = ranker.predict()
    fixer.fix(rank, n)
    return rank


@tf.function
def rankit(ranker):
    rank = ranker.predict()
    return rank


@tf.function
def fixit(fixer, rank, n):
    fixer.fix(rank, n)


@tf.function
def train(manager):
    manager.fit()

In [10]:
seed = 2987429
proc = AdultNoCorrProcessor(seed)
print(proc.ytrain.shape)
manager = ModelManager(proc.Xtrain, proc.ytrain, LogReg(proc, 2))
manager.fit()
print("LogReg")
print("On Training Clean\n", classification_report(tf.argmax(proc.ytrain, axis=1).numpy(), manager.predict(proc.Xtrain).numpy()))
print("On Testing\n", classification_report(tf.argmax(proc.ytest, axis=1).numpy(), manager.predict(proc.Xtest).numpy()))


(26048,)


AssertionError: labels should be onehot encoded but got shape (26048,)

In [11]:
# Init
manager0 = ModelManager(proc.Xtrain, proc.ytrain, LogReg(proc, proc.ytrain.shape[1]))
manager0.fit()
proc.post_init(manager0)

IndexError: list index out of range

In [9]:
# No retraining IUP
manager = ModelManager(proc.Xtrain, proc.ytrain, LogReg(proc, proc.ytrain.shape[1]))
manager.model.set_weights(manager0.model.get_weights())
manager.delta = tf.Variable(manager0.delta.value(), name="delta")
manager_check = ModelManager(proc.Xtrain, proc.ytrain, LogReg(proc, proc.ytrain.shape[1]))
manager_check.model.set_weights(manager0.model.get_weights())

K = 2000
corrsel = tf.cast(tf.ones(proc.ytrain.shape[0]), dtype='bool')
ranker = InfluenceRanker(manager=manager, on=proc.complain)
fixer = AutoFixer(manager, corrsel, K)

@tf.function
def change_model(manager, i):
    egrad = manager.egrads(range_=[i])
    ihvps = manager.iHvp(egrad)
    n = tf.reduce_sum(manager.delta)
    for var, ihvp in zip(manager.variables, ihvps):
        var.assign_add(ihvp / n)

# AQ is the margin of male - female
AQs = []
weighted_f1 = []
rank_time_iup = 0
model_time_iup = 0
for k in trange(0, K):
    start = time.time()
    scores = rankit(ranker)
    ranks = tf.argsort(-scores)
    tf.print(len(ranks))
    tf.print(ranks[:20])
    middle = time.time()
    change_model(manager, ranks[0])
    end = time.time()
    
    rank_time_iup += middle - start
    model_time_iup += end - middle
    
    fixit(fixer, scores, 1)
    manager_check.delta = tf.Variable(manager.delta.value(), name="delta")
    manager_check.fit()
    _, AQ, _, _ = proc.complain(manager_check)
    f1 = f1_score(tf.argmax(proc.ytest, axis=1).numpy(), manager_check.predict(proc.Xtest).numpy(), average='weighted')
    AQs.append(float(AQ))
    weighted_f1.append(f1)

print("Rank_time:", rank_time_iup)
print("Model_time:", model_time_iup)

df_iup = pd.DataFrame({
    "Complain": np.array(AQs),
    "F1": np.array(weighted_f1),
    "K": np.arange(len(AQs)) + 1,
    "Method": np.repeat("IUp", len(AQs)),
})
alt.Chart(pd.concat([df_iup])).mark_line().encode(
    x = "K",
    y = "Complain",
    color = "Method"
)

100%|██████████| 2000/2000 [36:45<00:00,  1.10s/it]

Rank_time: 457.9867331981659
Model_time: 467.5381124019623


alt.Chart(...)

In [13]:
# Once
K = 2000
corrsel = tf.cast(tf.ones(proc.ytrain.shape[0]), dtype='bool')
manager = ModelManager(proc.Xtrain, proc.ytrain, LogReg(proc, proc.ytrain.shape[1]))
manager.model.set_weights(manager0.model.get_weights())
manager.delta = tf.Variable(manager0.delta.value(), name="delta")
ranker = InfluenceRanker(manager=manager, on=proc.complain)
fixer = AutoFixer(manager, corrsel, K)

AQs = []
weighted_f1 = []
rank_time_once = 0
model_time_once = 0
_, AQ, _, _ = proc.complain(manager)
f1 = f1_score(tf.argmax(proc.ytest, axis=1).numpy(), manager.predict(proc.Xtest).numpy(), average='weighted')
AQs.append(float(AQ))
weighted_f1.append(f1)

start = time.time()
rank_fix(ranker, fixer, K)
middle = time.time()
train(manager)
end = time.time()

rank_time_once += middle - start
model_time_once += end - middle

_, AQ, _, _ = proc.complain(manager)
f1 = f1_score(tf.argmax(proc.ytest, axis=1).numpy(), manager.predict(proc.Xtest).numpy(), average='weighted')
AQs.append(float(AQ))
weighted_f1.append(f1)

print("Rank_time:", rank_time_once)
print("Model_time:", model_time_once)

df_once = pd.DataFrame({
    "Complain": np.array(AQs),
    "F1": np.array(weighted_f1),
    "K": [1, 2000],
    "Method": np.repeat("Once", len(AQs)),
})
alt.Chart(pd.concat([df_once, df_iup])).mark_line().encode(
    x = "K",
    y = "Complain",
    color = "Method"
)

Rank_time: 5.198538064956665
Model_time: 5.325089931488037


NameError: name 'df_iup' is not defined

In [14]:
# Rain
manager = ModelManager(proc.Xtrain, proc.ytrain, LogReg(proc, proc.ytrain.shape[1]))
manager.model.set_weights(manager0.model.get_weights())
manager.delta = tf.Variable(manager0.delta.value(), name="delta")
ranker = InfluenceRanker(manager=manager, on=proc.complain)
fixer = AutoFixer(manager, corrsel, K)

AQs = []
weighted_f1 = []
rank_list = []
rank_time_rain = 0
model_time_rain = 0
_, AQ, _, _ = proc.complain(manager)
f1 = f1_score(tf.argmax(proc.ytest, axis=1).numpy(), manager.predict(proc.Xtest).numpy(), average='weighted')
AQs.append(float(AQ))
weighted_f1.append(f1)

step_size = 10
rain_k = int(np.ceil(K / step_size))
for k in trange(0, rain_k):
    nfix = min(step_size, K - step_size * k)
    assert nfix > 0

    start = time.time()
    rank = rank_fix(ranker, fixer, nfix)
    middle = time.time()
    train(manager)
    end = time.time()
    
    rank_list.append(rank.numpy())
    rank_time_rain += middle - start
    model_time_rain += end - middle

    _, AQ, _, _ = proc.complain(manager)
    f1 = f1_score(tf.argmax(proc.ytest, axis=1).numpy(), manager.predict(proc.Xtest).numpy(), average='weighted')
    AQs.append(float(AQ))
    weighted_f1.append(f1)

print("Rank_time:", rank_time_rain)
print("Model_time:", model_time_rain)

df_rain = pd.DataFrame({
    "Complain": np.array(AQs),
    "F1": np.array(weighted_f1),
    "K": [1] + list(range(step_size, K + step_size, step_size)),
    "Method": np.repeat("Rain", len(AQs)),
})
alt.Chart(pd.concat([df_rain, df_once, df_iup])).mark_line().encode(
    x = "K",
    y = "Complain",
    color = "Method"
)

100%|██████████| 200/200 [01:36<00:00,  2.07it/s]

Rank_time: 46.04503607749939
Model_time: 49.52215528488159


NameError: name 'df_iup' is not defined

In [14]:
# Once Improved
manager = ModelManager(proc.Xtrain, proc.ytrain, LogReg(proc, proc.ytrain.shape[1]))
manager.model.set_weights(manager0.model.get_weights())
manager.delta = tf.Variable(manager0.delta.value(), name="delta")
ranker = InfluenceRanker(manager=manager, on=proc.complain)
fixer = AutoFixer(manager, corrsel, K)
bpts = np.asarray([0, 400, 800, 1200, 1600, 2000])

AQs = []
weighted_f1 = []
rank_time_oncei = 0
model_time_oncei = 0
_, AQ, _, _ = proc.complain(manager)
f1 = f1_score(tf.argmax(proc.ytest, axis=1).numpy(), manager.predict(proc.Xtest).numpy(), average='weighted')
AQs.append(float(AQ))
weighted_f1.append(f1)

for nfix in bpts[1:] - bpts[:-1]:
    start = time.time()
    rank_fix(ranker, fixer, nfix)
    middle = time.time()
    train(manager)
    end = time.time()
    
    rank_time_oncei += middle - start
    model_time_oncei += end - middle

    _, AQ, _, _ = proc.complain(manager)
    f1 = f1_score(tf.argmax(proc.ytest, axis=1).numpy(), manager.predict(proc.Xtest).numpy(), average='weighted')
    AQs.append(float(AQ))
    weighted_f1.append(f1)

print("Rank_time:", rank_time_oncei)
print("Model_time:", model_time_oncei)

bpts[0] += 1
df_oncei = pd.DataFrame({
    "Complain": np.array(AQs),
    "F1": np.array(weighted_f1),
    "K": bpts,
    "Method": np.repeat("OnceI", len(AQs)),
})
alt.Chart(pd.concat([df_oncei, df_rain, df_once, df_iup])).mark_line().encode(
    x = "K",
    y = "Complain",
    color = "Method"
)

Rank_time: 1.8292059898376465
Model_time: 5.835071086883545


alt.Chart(...)

In [17]:
# IUPI
@tf.function
def change_model(manager, i):
    egrad = manager.egrads(range_=[i])
    ihvps = manager.iHvp(egrad)
    n = tf.reduce_sum(manager.delta)
    for var, ihvp in zip(manager.variables, ihvps):
        var.assign_add(ihvp / n)

from tqdm import tqdm
manager = ModelManager(proc.Xtrain, proc.ytrain, LogReg(proc, proc.ytrain.shape[1]))
manager.model.set_weights(manager0.model.get_weights())
manager.delta = tf.Variable(manager0.delta.value(), name="delta")
ranker = InfluenceRanker(manager=manager, on=proc.complain)
fixer = AutoFixer(manager, corrsel, K)
manager_check = ModelManager(proc.Xtrain, proc.ytrain, LogReg(proc, proc.ytrain.shape[1]))
manager_check.model.set_weights(manager0.model.get_weights())

AQs = []
weighted_f1 = []
rank_time_iupi = 0
model_time_iupi = 0
positions = []
trainingKs = []
pbar = tqdm(total=K)
n = proc.ytrain.shape[0]
while pbar.n < K:
    start = time.time()
    scores = rankit(ranker)
    ranks = tf.argsort(-scores)

    indices = manager.to_original_index(ranks)
    i2 = np.empty(n)
    i2[:] = np.nan
    i2[indices] = np.arange(len(indices)) + 1
    positions.append(i2)

    if len(positions) > 1:
        topidx = tf.argsort(positions[-1])[:10]
        chs = (positions[-1] - positions[-2])[topidx.numpy()]
        need_train = (chs < -500).sum() > 3
    else:
        need_train = True

    middle = time.time()
    if need_train:
        trainingKs.append(pbar.n)
        train(manager)
        rank_fix(ranker, fixer, 10)
        pbar.update(10)
    else:
        change_model(manager, ranks[0])
        fixer.fix(scores, 1)
        pbar.update(1)
    end = time.time()
        
    rank_time_iupi += middle - start
    model_time_iupi += end - middle
    
    manager_check.delta = tf.Variable(manager.delta.value(), name="delta")
    manager_check.fit()
    _, AQ, _, _ = proc.complain(manager_check)
    f1 = f1_score(tf.argmax(proc.ytest, axis=1).numpy(), manager_check.predict(proc.Xtest).numpy(), average='weighted')
    AQs.append(float(AQ))
    weighted_f1.append(f1)

df_iupi = pd.DataFrame({
    "Complain": np.array(AQs),
    "F1": np.array(weighted_f1),
    "K": np.arange(len(AQs)) + 1,
    "Method": np.repeat("IUpI", len(AQs)),
})
alt.Chart(pd.concat([df_iupi, df_rain, df_once])).mark_line().encode(
    x = "K",
    y = "Complain",
    color = "Method"
)



  0%|          | 0/2000 [00:00<?, ?it/s]

  0%|          | 10/2000 [00:06<21:36,  1.53it/s]

  1%|          | 11/2000 [00:09<40:19,  1.22s/it]

  1%|          | 12/2000 [00:10<37:46,  1.14s/it]

  1%|          | 13/2000 [00:10<34:01,  1.03s/it]

  1%|          | 14/2000 [00:11<33:47,  1.02s/it]

  1%|          | 15/2000 [00:12<34:17,  1.04s/it]

  1%|          | 16/2000 [00:13<32:27,  1.02it/s]

  1%|          | 17/2000 [00:14<28:04,  1.18it/s]/home/yejia/.local/share/virtualenvs/Rain-Experiments-jsPNbhrB/lib/python3.7/site-packages/ipykernel_launcher.py:41: RuntimeWarning: invalid value encountered in less


  1%|          | 18/2000 [00:15<36:11,  1.10s/it]

  1%|          | 19/2000 [00:16<31:40,  1.04it/s]

  1%|          | 20/2000 [00:17<28:50,  1.14it/s]

  1%|          | 21/2000 [00:18<32:07,  1.03it/s]

  1%|          | 22/2000 [00:19<31:01,  1.06it/s]

  1%|          | 23/2000 [00:20<32:01,  1.03it/s]

  1%|          | 24/2000 [00:20<27:45,  1.19it/s]

  1%|▏         | 25/2000



  5%|▌         | 106/2000 [01:51<36:42,  1.16s/it]/home/yejia/.local/share/virtualenvs/Rain-Experiments-jsPNbhrB/lib/python3.7/site-packages/ipykernel_launcher.py:41: RuntimeWarning: invalid value encountered in less


  5%|▌         | 107/2000 [01:52<40:56,  1.30s/it]

  5%|▌         | 108/2000 [01:53<39:45,  1.26s/it]

  5%|▌         | 109/2000 [01:54<34:29,  1.09s/it]

  6%|▌         | 110/2000 [01:55<29:14,  1.08it/s]/home/yejia/.local/share/virtualenvs/Rain-Experiments-jsPNbhrB/lib/python3.7/site-packages/ipykernel_launcher.py:41: RuntimeWarning: invalid value encountered in less


  6%|▌         | 111/2000 [01:56<28:53,  1.09it/s]

  6%|▌         | 112/2000 [01:57<30:36,  1.03it/s]

  6%|▌         | 113/2000 [01:58<33:29,  1.06s/it]

  6%|▌         | 114/2000 [01:59<30:17,  1.04it/s]

  6%|▌         | 115/2000 [02:00<30:26,  1.03it/s]

  6%|▌         | 116/2000 [02:00<27:05,  1.16it/s]

  6%|▌         | 117/2000 [02:01<28:02,  1.12it/s]

  6%|▌         | 118/2000 [02:02<24:39, 



 10%|▉         | 198/2000 [03:21<26:24,  1.14it/s]/home/yejia/.local/share/virtualenvs/Rain-Experiments-jsPNbhrB/lib/python3.7/site-packages/ipykernel_launcher.py:41: RuntimeWarning: invalid value encountered in less


 10%|▉         | 199/2000 [03:21<24:25,  1.23it/s]

 10%|█         | 200/2000 [03:22<22:05,  1.36it/s]

 10%|█         | 201/2000 [03:23<23:02,  1.30it/s]

 10%|█         | 202/2000 [03:24<24:09,  1.24it/s]

 10%|█         | 203/2000 [03:25<25:32,  1.17it/s]

 10%|█         | 204/2000 [03:26<26:51,  1.11it/s]

 10%|█         | 205/2000 [03:26<25:13,  1.19it/s]

 10%|█         | 206/2000 [03:28<29:47,  1.00it/s]

 10%|█         | 207/2000 [03:28<28:36,  1.04it/s]

 10%|█         | 208/2000 [03:30<32:39,  1.09s/it]

 10%|█         | 209/2000 [03:31<30:51,  1.03s/it]

 10%|█         | 210/2000 [03:31<27:45,  1.07it/s]

 11%|█         | 211/2000 [03:33<30:00,  1.01s/it]

 11%|█         | 212/2000 [03:34<31:32,  1.06s/it]

 11%|█         | 213/2000 [03:35<32:21,  1.09s/it]




 14%|█▍        | 289/2000 [04:45<21:25,  1.33it/s]

 14%|█▍        | 290/2000 [04:46<26:44,  1.07it/s]/home/yejia/.local/share/virtualenvs/Rain-Experiments-jsPNbhrB/lib/python3.7/site-packages/ipykernel_launcher.py:41: RuntimeWarning: invalid value encountered in less


 15%|█▍        | 291/2000 [04:47<25:59,  1.10it/s]

 15%|█▍        | 292/2000 [04:47<22:22,  1.27it/s]

 15%|█▍        | 293/2000 [04:48<20:13,  1.41it/s]

 15%|█▍        | 294/2000 [04:49<26:37,  1.07it/s]

 15%|█▍        | 295/2000 [04:50<24:53,  1.14it/s]/home/yejia/.local/share/virtualenvs/Rain-Experiments-jsPNbhrB/lib/python3.7/site-packages/ipykernel_launcher.py:41: RuntimeWarning: invalid value encountered in less


 15%|█▍        | 296/2000 [04:52<29:50,  1.05s/it]

 15%|█▍        | 297/2000 [04:52<27:11,  1.04it/s]/home/yejia/.local/share/virtualenvs/Rain-Experiments-jsPNbhrB/lib/python3.7/site-packages/ipykernel_launcher.py:41: RuntimeWarning: invalid value encountered in less


 15%|█▍        | 298/2000 [04



 17%|█▋        | 347/2000 [05:41<29:27,  1.07s/it]

 17%|█▋        | 348/2000 [05:42<27:54,  1.01s/it]

 17%|█▋        | 349/2000 [05:43<29:18,  1.07s/it]/home/yejia/.local/share/virtualenvs/Rain-Experiments-jsPNbhrB/lib/python3.7/site-packages/ipykernel_launcher.py:41: RuntimeWarning: invalid value encountered in less


 18%|█▊        | 350/2000 [05:44<24:59,  1.10it/s]

 18%|█▊        | 351/2000 [05:44<21:59,  1.25it/s]

 18%|█▊        | 352/2000 [05:45<21:45,  1.26it/s]/home/yejia/.local/share/virtualenvs/Rain-Experiments-jsPNbhrB/lib/python3.7/site-packages/ipykernel_launcher.py:41: RuntimeWarning: invalid value encountered in less


 18%|█▊        | 353/2000 [05:46<19:32,  1.40it/s]

 18%|█▊        | 354/2000 [05:47<24:03,  1.14it/s]

 18%|█▊        | 355/2000 [05:48<24:32,  1.12it/s]

 18%|█▊        | 356/2000 [05:49<26:24,  1.04it/s]

 18%|█▊        | 357/2000 [05:50<28:39,  1.05s/it]

 18%|█▊        | 358/2000 [05:52<32:36,  1.19s/it]

 18%|█▊        | 359/2000 [05:53<35:11, 

...



 86%|████████▌ | 1711/2000 [30:16<05:52,  1.22s/it]/home/yejia/.local/share/virtualenvs/Rain-Experiments-jsPNbhrB/lib/python3.7/site-packages/ipykernel_launcher.py:41: RuntimeWarning: invalid value encountered in less


 86%|████████▌ | 1712/2000 [30:18<06:11,  1.29s/it]/home/yejia/.local/share/virtualenvs/Rain-Experiments-jsPNbhrB/lib/python3.7/site-packages/ipykernel_launcher.py:41: RuntimeWarning: invalid value encountered in less


 86%|████████▌ | 1713/2000 [30:19<06:09,  1.29s/it]/home/yejia/.local/share/virtualenvs/Rain-Experiments-jsPNbhrB/lib/python3.7/site-packages/ipykernel_launcher.py:41: RuntimeWarning: invalid value encountered in less


 86%|████████▌ | 1714/2000 [30:21<06:20,  1.33s/it]

 86%|████████▌ | 1715/2000 [30:22<06:32,  1.38s/it]

 86%|████████▌ | 1716/2000 [30:23<05:55,  1.25s/it]

 86%|████████▌ | 1717/2000 [30:25<06:24,  1.36s/it]/home/yejia/.local/share/virtualenvs/Rain-Experiments-jsPNbhrB/lib/python3.7/site-packages/ipykernel_launcher.py:41: RuntimeWarn



 91%|█████████ | 1824/2000 [32:31<03:40,  1.25s/it]/home/yejia/.local/share/virtualenvs/Rain-Experiments-jsPNbhrB/lib/python3.7/site-packages/ipykernel_launcher.py:41: RuntimeWarning: invalid value encountered in less


 91%|█████████▏| 1825/2000 [32:32<04:02,  1.39s/it]/home/yejia/.local/share/virtualenvs/Rain-Experiments-jsPNbhrB/lib/python3.7/site-packages/ipykernel_launcher.py:41: RuntimeWarning: invalid value encountered in less


 91%|█████████▏| 1826/2000 [32:34<04:03,  1.40s/it]/home/yejia/.local/share/virtualenvs/Rain-Experiments-jsPNbhrB/lib/python3.7/site-packages/ipykernel_launcher.py:41: RuntimeWarning: invalid value encountered in less


 91%|█████████▏| 1827/2000 [32:35<03:46,  1.31s/it]

 91%|█████████▏| 1828/2000 [32:36<03:10,  1.11s/it]/home/yejia/.local/share/virtualenvs/Rain-Experiments-jsPNbhrB/lib/python3.7/site-packages/ipykernel_launcher.py:41: RuntimeWarning: invalid value encountered in less


 91%|█████████▏| 1829/2000 [32:37<03:45,  1.32s/it]/home/yejia/.l



 94%|█████████▎| 1870/2000 [33:28<02:37,  1.21s/it]/home/yejia/.local/share/virtualenvs/Rain-Experiments-jsPNbhrB/lib/python3.7/site-packages/ipykernel_launcher.py:41: RuntimeWarning: invalid value encountered in less


 94%|█████████▎| 1871/2000 [33:30<03:13,  1.50s/it]

 94%|█████████▎| 1872/2000 [33:32<03:22,  1.58s/it]

 94%|█████████▎| 1873/2000 [33:34<03:29,  1.65s/it]

 94%|█████████▎| 1874/2000 [33:35<03:13,  1.54s/it]

 94%|█████████▍| 1875/2000 [33:37<03:36,  1.73s/it]/home/yejia/.local/share/virtualenvs/Rain-Experiments-jsPNbhrB/lib/python3.7/site-packages/ipykernel_launcher.py:41: RuntimeWarning: invalid value encountered in less


 94%|█████████▍| 1876/2000 [33:39<03:16,  1.58s/it]

 94%|█████████▍| 1877/2000 [33:40<03:02,  1.49s/it]/home/yejia/.local/share/virtualenvs/Rain-Experiments-jsPNbhrB/lib/python3.7/site-packages/ipykernel_launcher.py:41: RuntimeWarning: invalid value encountered in less


 94%|█████████▍| 1878/2000 [33:41<02:43,  1.34s/it]

 94%|█████████▍| 187



 96%|█████████▋| 1927/2000 [34:42<01:25,  1.17s/it]/home/yejia/.local/share/virtualenvs/Rain-Experiments-jsPNbhrB/lib/python3.7/site-packages/ipykernel_launcher.py:41: RuntimeWarning: invalid value encountered in less


 96%|█████████▋| 1928/2000 [34:43<01:17,  1.07s/it]/home/yejia/.local/share/virtualenvs/Rain-Experiments-jsPNbhrB/lib/python3.7/site-packages/ipykernel_launcher.py:41: RuntimeWarning: invalid value encountered in less


 96%|█████████▋| 1929/2000 [34:44<01:20,  1.13s/it]/home/yejia/.local/share/virtualenvs/Rain-Experiments-jsPNbhrB/lib/python3.7/site-packages/ipykernel_launcher.py:41: RuntimeWarning: invalid value encountered in less


 96%|█████████▋| 1930/2000 [34:45<01:17,  1.11s/it]/home/yejia/.local/share/virtualenvs/Rain-Experiments-jsPNbhrB/lib/python3.7/site-packages/ipykernel_launcher.py:41: RuntimeWarning: invalid value encountered in less


 97%|█████████▋| 1931/2000 [34:47<01:38,  1.43s/it]/home/yejia/.local/share/virtualenvs/Rain-Experiments-jsPNbhrB/lib/

alt.Chart(...)

In [112]:
# No retraining IUP accelerate
manager = ModelManager(proc.Xtrain, proc.ytrain, LogReg(proc, proc.ytrain.shape[1]))
manager.model.set_weights(manager0.model.get_weights())
manager.delta = tf.Variable(manager0.delta.value(), name="delta")
manager_check = ModelManager(proc.Xtrain, proc.ytrain, LogReg(proc, proc.ytrain.shape[1]))
manager_check.model.set_weights(manager0.model.get_weights())

K = 2000
step_size = 10
corrsel = tf.cast(tf.ones(proc.ytrain.shape[0]), dtype='bool')
ranker = InfluenceRanker(manager=manager, on=proc.complain)
fixer = AutoFixer(manager, corrsel, K)

@tf.function
def change_model(manager, d): # d is a list
    egrads = manager.egrads(range_=d)
    egrad = [tf.reduce_sum(j, axis=0) for j in egrads]
    ihvps = manager.iHvp(egrad)
    n = tf.reduce_sum(manager.delta)
    for var, ihvp in zip(manager.variables, ihvps):
        var.assign_add(ihvp / n)

# AQ is the margin of male - female
AQs = []
weighted_f1 = []
AQs_re = []
weighted_f1_re = []
rank_time_iupac = 0
model_time_iupac = 0
for k in trange(0, int(K / step_size)):
    start = time.time()
    scores = rankit(ranker)
    ranks = tf.argsort(-scores)
    middle = time.time()
    change_model(manager, ranks[:step_size])
    end = time.time()
    
    rank_time_iupac += middle - start
    model_time_iupac += end - middle
    
    fixit(fixer, scores, step_size)  # update delta
    manager_check.delta = tf.Variable(manager.delta.value(), name="delta")
    manager_check.fit()
    _, AQ, _, _ = proc.complain(manager_check)
    f1 = f1_score(tf.argmax(proc.ytest, axis=1).numpy(), manager_check.predict(proc.Xtest).numpy(), average='weighted')
    AQs_re.append(float(AQ))
    weighted_f1_re.append(f1)
    _, AQ, _, _ = proc.complain(manager)
    f1 = f1_score(tf.argmax(proc.ytest, axis=1).numpy(), manager.predict(proc.Xtest).numpy(), average='weighted')
    AQs.append(float(AQ))
    weighted_f1.append(f1)

print("Rank_time:", rank_time_iupac)
print("Model_time:", model_time_iupac)

df_iupac_re = pd.DataFrame({
    "Complain": np.array(AQs_re),
    "F1": np.array(weighted_f1_re),
    "K": np.arange(len(AQs_re)) + 1,
    "Method": np.repeat("IUpAC_Re", len(AQs_re)),
})
df_iupac = pd.DataFrame({
    "Complain": np.array(AQs),
    "F1": np.array(weighted_f1),
    "K": np.arange(len(AQs)) + 1,
    "Method": np.repeat("IUpAC", len(AQs)),
})
alt.Chart(pd.concat([df_iupac, df_iupac_re])).mark_line().encode(
    x = "K",
    y = "Complain",
    color = "Method"
)





  0%|          | 0/200 [00:00<?, ?it/s]



  0%|          | 1/200 [00:02<09:44,  2.94s/it]



  1%|          | 2/200 [00:04<08:34,  2.60s/it]



  2%|▏         | 3/200 [00:06<07:27,  2.27s/it]



  2%|▏         | 4/200 [00:07<06:31,  2.00s/it]



  2%|▎         | 5/200 [00:08<05:23,  1.66s/it]



  3%|▎         | 6/200 [00:09<04:59,  1.54s/it]



  4%|▎         | 7/200 [00:11<05:02,  1.57s/it]



  4%|▍         | 8/200 [00:12<04:31,  1.41s/it]



  4%|▍         | 9/200 [00:13<04:30,  1.42s/it]



  5%|▌         | 10/200 [00:14<04:03,  1.28s/it]



  6%|▌         | 11/200 [00:15<03:43,  1.18s/it]



  6%|▌         | 12/200 [00:16<03:37,  1.16s/it]



  6%|▋         | 13/200 [00:18<03:37,  1.16s/it]



  7%|▋         | 14/200 [00:19<03:32,  1.14s/it]



  8%|▊         | 15/200 [00:20<03:46,  1.23s/it]



  8%|▊         | 16/200 [00:21<03:22,  1.10s/it]



  8%|▊         | 17/200 [00:22<03:13,  1.06s/it]



  9%|▉         | 18/200 [00:23<03:31,  1.16s/it]



 10%|▉         | 19/200 [0

Rank_time: 45.617143392562866
Model_time: 46.0634605884552


alt.Chart(...)

In [19]:
K = 2000
step_size = 10

@tf.function
def change_model(manager, d): # d is a list
    egrads = manager.egrads(range_=d)
    egrad = [tf.reduce_sum(j, axis=0) for j in egrads]
    ihvps = manager.iHvp(egrad)
    n = tf.reduce_sum(manager.delta)
    for var, ihvp in zip(manager.variables, ihvps):
        var.assign_add(ihvp / n)

In [140]:
# No retraining IUP accelerate
manager = ModelManager(proc.Xtrain, proc.ytrain, LogReg(proc, proc.ytrain.shape[1]))
manager.model.set_weights(manager0.model.get_weights())
manager.delta = tf.Variable(manager0.delta.value(), name="delta")
manager_check = ModelManager(proc.Xtrain, proc.ytrain, LogReg(proc, proc.ytrain.shape[1]))
manager_check.model.set_weights(manager0.model.get_weights())

corrsel = tf.cast(tf.ones(proc.ytrain.shape[0]), dtype='bool')
ranker = InfluenceRanker(manager=manager, on=proc.complain)
fixer = AutoFixer(manager, corrsel, K)

# AQ is the margin of male - female
AQs = []
weighted_f1 = []
AQs_re = []
weighted_f1_re = []
delta = tf.Variable(manager0.delta.value(), name="delta")
rank_time_iupac_init = 0
model_time_iupac_init = 0
for k in trange(0, int(K / step_size)):
    start = time.time()
    scores = rankit(ranker)
    ranks = tf.argsort(-scores)
    middle = time.time()
    
    fixit(fixer, scores, step_size)  # update delta
    delta = manager.delta.value()  # store
    manager = ModelManager(proc.Xtrain, proc.ytrain, LogReg(proc, proc.ytrain.shape[1]))
    manager.model.set_weights(manager0.model.get_weights())  # clear
    manager.delta = tf.Variable(manager0.delta.value(), name="delta")  # clear
    change_model(manager, fixer.deletions.numpy().tolist()[:(k+1)*step_size])  # update
    manager.delta = tf.Variable(delta, name="delta") # reassign
#     change_model(manager, ranks[:step_size])
    end = time.time()
    
    ranker = InfluenceRanker(manager=manager, on=proc.complain)
    fixer.manager = manager
    rank_time_iupac_init += middle - start
    model_time_iupac_init += end - middle
    
    manager_check.delta = tf.Variable(manager.delta.value(), name="delta")
    manager_check.fit()
    _, AQ, _, _ = proc.complain(manager_check)
    f1 = f1_score(tf.argmax(proc.ytest, axis=1).numpy(), manager_check.predict(proc.Xtest).numpy(), average='weighted')
    AQs_re.append(float(AQ))
    weighted_f1_re.append(f1)
    _, AQ, _, _ = proc.complain(manager)
    f1 = f1_score(tf.argmax(proc.ytest, axis=1).numpy(), manager.predict(proc.Xtest).numpy(), average='weighted')
    AQs.append(float(AQ))
    weighted_f1.append(f1)

print("Rank_time:", rank_time_iupac_init)
print("Model_time:", model_time_iupac_init)

df_iupac_init_re = pd.DataFrame({
    "Complain": np.array(AQs_re),
    "F1": np.array(weighted_f1_re),
    "K": np.arange(len(AQs_re)) + 1,
    "Method": np.repeat("IUpAC_Init_Re", len(AQs_re)),
})
df_iupac_init = pd.DataFrame({
    "Complain": np.array(AQs),
    "F1": np.array(weighted_f1),
    "K": np.arange(len(AQs)) + 1,
    "Method": np.repeat("IUpAC_Init", len(AQs)),
})
alt.Chart(pd.concat([df_iupac_init, df_iupac_init_re])).mark_line().encode(
    x = "K",
    y = "Complain",
    color = "Method"
)











  0%|          | 0/200 [00:00<?, ?it/s]









  0%|          | 1/200 [00:04<14:35,  4.40s/it]









  1%|          | 2/200 [00:09<15:24,  4.67s/it]









  2%|▏         | 3/200 [00:14<15:35,  4.75s/it]









  2%|▏         | 4/200 [00:19<15:25,  4.72s/it]









  2%|▎         | 5/200 [00:23<14:22,  4.43s/it]









  3%|▎         | 6/200 [00:27<14:04,  4.35s/it]









  4%|▎         | 7/200 [00:32<14:30,  4.51s/it]









  4%|▍         | 8/200 [00:36<13:52,  4.34s/it]









  4%|▍         | 9/200 [00:40<13:55,  4.37s/it]









  5%|▌         | 10/200 [00:44<13:21,  4.22s/it]









  6%|▌         | 11/200 [00:48<12:48,  4.07s/it]









  6%|▌         | 12/200 [00:52<12:43,  4.06s/it]









  6%|▋         | 13/200 [00:56<12:45,  4.09s/it]









  7%|▋         | 14/200 [01:01<13:19,  4.30s/it]









  8%|▊         | 15/200 [01:05<13:06,  4.25s/it]









  8%|▊         | 16/200 [01:12<15:57,  5.21s/it]









  8%|▊         | 17/20

 92%|█████████▎| 185/200 [12:50<00:40,  2.67s/it]









 93%|█████████▎| 186/200 [12:53<00:37,  2.68s/it]









 94%|█████████▎| 187/200 [12:56<00:36,  2.80s/it]









 94%|█████████▍| 188/200 [12:59<00:33,  2.79s/it]









 94%|█████████▍| 189/200 [13:02<00:30,  2.77s/it]









 95%|█████████▌| 190/200 [13:05<00:29,  2.93s/it]









 96%|█████████▌| 191/200 [13:08<00:27,  3.06s/it]









 96%|█████████▌| 192/200 [13:11<00:24,  3.01s/it]









 96%|█████████▋| 193/200 [13:14<00:19,  2.85s/it]









 97%|█████████▋| 194/200 [13:17<00:17,  2.85s/it]









 98%|█████████▊| 195/200 [13:20<00:14,  2.98s/it]









 98%|█████████▊| 196/200 [13:23<00:11,  2.94s/it]









 98%|█████████▊| 197/200 [13:25<00:08,  2.77s/it]









 99%|█████████▉| 198/200 [13:28<00:05,  2.87s/it]









100%|█████████▉| 199/200 [13:31<00:02,  2.74s/it]









100%|██████████| 200/200 [13:33<00:00,  4.07s/it]

Rank_time: 237.39967441558838
Model_time: 342.0709705352783


NameError: name 'df_inpac' is not defined

In [26]:
# IUPI AC
from tqdm import tqdm
manager = ModelManager(proc.Xtrain, proc.ytrain, LogReg(proc, proc.ytrain.shape[1]))
manager.model.set_weights(manager0.model.get_weights())
manager.delta = tf.Variable(manager0.delta.value(), name="delta")
ranker = InfluenceRanker(manager=manager, on=proc.complain)
fixer = AutoFixer(manager, corrsel, K)
manager_check = ModelManager(proc.Xtrain, proc.ytrain, LogReg(proc, proc.ytrain.shape[1]))
manager_check.model.set_weights(manager0.model.get_weights())

AQs = []
weighted_f1 = []
AQs_re = []
weighted_f1_re = []
rank_time_iupi_s10 = 0
model_time_iupi_s10 = 0
positions = []
trainingKs = []
# delta = tf.Variable(manager0.delta.value(), name="delta")
pbar = tqdm(total=K)
n = proc.ytrain.shape[0]
while pbar.n < K:
    start = time.time()
    scores = rankit(ranker)
    ranks = tf.argsort(-scores)

    indices = manager.to_original_index(ranks)
    i2 = np.empty(n)
    i2[:] = np.nan
    i2[indices] = np.arange(len(indices)) + 1
    positions.append(i2)

    if len(positions) > 1:
        topidx = tf.argsort(positions[-1])[:10]
        chs = (positions[-1] - positions[-2])[topidx.numpy()]
        need_train = (chs < -500).sum() > 3
    else:
        need_train = True

#     manager = ModelManager(proc.Xtrain, proc.ytrain, LogReg(proc, proc.ytrain.shape[1]))
#     manager.model.set_weights(manager0.model.get_weights())  # clear
#     manager.delta = tf.Variable(manager0.delta.value(), name="delta")  # clear
    if need_train:
        trainingKs.append(pbar.n)
#         manager.delta = tf.Variable(delta, name="delta") # reassign
#         fixer.manager = manager
        fixit(fixer, scores, step_size)
        middle = time.time()
        train(manager)
        end = time.time()
#         delta = manager.delta.value()  # store
#         ranker = InfluenceRanker(manager=manager, on=proc.complain)
        pbar.update(10)
    else:
#         print(fixer.deletions.numpy().tolist()[:pbar.n] + ranks[:step_size].numpy().tolist())
#         change_model(manager, fixer.deletions.numpy().tolist()[:pbar.n] + ranks[:step_size].numpy().tolist())  # update
#         manager.delta = tf.Variable(delta, name="delta") # reassign
        middle = time.time()
        change_model(manager, ranks[:step_size])
        end = time.time()
#         fixer.manager = manager
        fixit(fixer, scores, step_size)  # update delta
#         delta = manager.delta.value()  # store
#         ranker = InfluenceRanker(manager=manager, on=proc.complain)
        pbar.update(10)
    
    rank_time_iupi_s10 += middle - start
    model_time_iupi_s10 += end - middle
    
    manager_check.delta = tf.Variable(manager.delta.value(), name="delta")
    manager_check.fit()
    _, AQ, _, _ = proc.complain(manager_check)
    f1 = f1_score(tf.argmax(proc.ytest, axis=1).numpy(), manager_check.predict(proc.Xtest).numpy(), average='weighted')
    AQs_re.append(float(AQ))
    weighted_f1_re.append(f1)
    _, AQ, _, _ = proc.complain(manager)
    f1 = f1_score(tf.argmax(proc.ytest, axis=1).numpy(), manager.predict(proc.Xtest).numpy(), average='weighted')
    AQs.append(float(AQ))
    weighted_f1.append(f1)

print("Rank_time:", rank_time_iupi_s10)
print("Model_time:", model_time_iupi_s10)

df_iupi_s10_re = pd.DataFrame({
    "Complain": np.array(AQs_re),
    "F1": np.array(weighted_f1_re),
    "K": (np.arange(len(AQs_re)) + 1) * 10,
    "Method": np.repeat("IUpI_Step10_Re", len(AQs_re)),
})
df_iupi_s10 = pd.DataFrame({
    "Complain": np.array(AQs),
    "F1": np.array(weighted_f1),
    "K": (np.arange(len(AQs)) + 1) * 10,
    "Method": np.repeat("IUpI_Step10", len(AQs)),
})
alt.Chart(pd.concat([df_iupi_s10, df_iupi_s10_re, df_iupi])).mark_line().encode(
    x = "K",
    y = "Complain",
    color = "Method"
)







  0%|          | 0/2000 [00:00<?, ?it/s]





  0%|          | 10/2000 [00:05<17:23,  1.91it/s]





  1%|          | 20/2000 [00:07<14:35,  2.26it/s]/home/yejia/.local/share/virtualenvs/Rain-Experiments-jsPNbhrB/lib/python3.7/site-packages/ipykernel_launcher.py:36: RuntimeWarning: invalid value encountered in less






  2%|▏         | 30/2000 [00:08<11:11,  2.94it/s]





  2%|▏         | 40/2000 [00:09<08:53,  3.67it/s]





  2%|▎         | 50/2000 [00:11<07:52,  4.13it/s]





  3%|▎         | 60/2000 [00:12<06:29,  4.99it/s]





  4%|▎         | 70/2000 [00:13<05:33,  5.79it/s]





  4%|▍         | 80/2000 [00:15<05:30,  5.81it/s]





  4%|▍         | 90/2000 [00:17<05:19,  5.98it/s]





  5%|▌         | 100/2000 [00:18<04:46,  6.63it/s]





  6%|▌         | 110/2000 [00:19<04:40,  6.73it/s]/home/yejia/.local/share/virtualenvs/Rain-Experiments-jsPNbhrB/lib/python3.7/site-packages/ipykernel_launcher.py:36: RuntimeWarning: invalid value encountered in less






  6%|▌ 







 33%|███▎      | 660/2000 [01:29<03:26,  6.50it/s]/home/yejia/.local/share/virtualenvs/Rain-Experiments-jsPNbhrB/lib/python3.7/site-packages/ipykernel_launcher.py:36: RuntimeWarning: invalid value encountered in less






 34%|███▎      | 670/2000 [01:30<03:24,  6.51it/s]





 34%|███▍      | 680/2000 [01:32<03:05,  7.11it/s]





 34%|███▍      | 690/2000 [01:33<03:03,  7.15it/s]/home/yejia/.local/share/virtualenvs/Rain-Experiments-jsPNbhrB/lib/python3.7/site-packages/ipykernel_launcher.py:36: RuntimeWarning: invalid value encountered in less






 35%|███▌      | 700/2000 [01:34<02:58,  7.27it/s]/home/yejia/.local/share/virtualenvs/Rain-Experiments-jsPNbhrB/lib/python3.7/site-packages/ipykernel_launcher.py:36: RuntimeWarning: invalid value encountered in less






 36%|███▌      | 710/2000 [01:35<02:39,  8.11it/s]/home/yejia/.local/share/virtualenvs/Rain-Experiments-jsPNbhrB/lib/python3.7/site-packages/ipykernel_launcher.py:36: RuntimeWarning: invalid value encountered in l

 60%|██████    | 1200/2000 [02:47<01:58,  6.75it/s]





 60%|██████    | 1210/2000 [02:48<01:49,  7.19it/s]/home/yejia/.local/share/virtualenvs/Rain-Experiments-jsPNbhrB/lib/python3.7/site-packages/ipykernel_launcher.py:36: RuntimeWarning: invalid value encountered in less






 61%|██████    | 1220/2000 [02:50<01:47,  7.27it/s]/home/yejia/.local/share/virtualenvs/Rain-Experiments-jsPNbhrB/lib/python3.7/site-packages/ipykernel_launcher.py:36: RuntimeWarning: invalid value encountered in less






 62%|██████▏   | 1230/2000 [02:51<01:43,  7.41it/s]/home/yejia/.local/share/virtualenvs/Rain-Experiments-jsPNbhrB/lib/python3.7/site-packages/ipykernel_launcher.py:36: RuntimeWarning: invalid value encountered in less






 62%|██████▏   | 1240/2000 [02:52<01:33,  8.15it/s]





 62%|██████▎   | 1250/2000 [02:53<01:39,  7.54it/s]/home/yejia/.local/share/virtualenvs/Rain-Experiments-jsPNbhrB/lib/python3.7/site-packages/ipykernel_launcher.py:36: RuntimeWarning: invalid value encountered in l

Rank_time: 46.36125349998474
Model_time: 52.456286668777466


alt.Chart(...)

In [208]:
with tf.GradientTape() as tape:
    loss_value = manager.loss()
    # Calculate the actual gradients
    grads = tape.gradient(loss_value, manager.variables)
grads

[<tf.Tensor: shape=(17, 2), dtype=float32, numpy=
 array([[-5.2069715e-04,  5.2075274e-04],
        [ 5.7361030e-04, -5.7357003e-04],
        [ 4.1736308e-05, -4.1706742e-05],
        [-6.7156827e-05,  6.7186098e-05],
        [-3.2903081e-05,  3.2740987e-05],
        [-4.3860578e-05,  4.3771714e-05],
        [ 3.1282092e-04, -3.1264970e-04],
        [ 7.3869247e-05, -7.3912226e-05],
        [ 5.1492356e-05, -5.1363033e-05],
        [ 1.5479974e-04, -1.5498759e-04],
        [ 1.5847963e-04, -1.5831264e-04],
        [-2.5306415e-04,  2.5322911e-04],
        [ 1.8601803e-04, -1.8608809e-04],
        [-1.6530690e-05,  1.6413800e-05],
        [ 1.5627850e-04, -1.5628294e-04],
        [ 6.4688902e-07, -6.2975414e-07],
        [-1.7840666e-05,  1.7854094e-05]], dtype=float32)>,
 <tf.Tensor: shape=(2,), dtype=float32, numpy=array([ 0.00020252, -0.00020251], dtype=float32)>]

In [186]:
tf.reduce_sum(manager.delta)

<tf.Tensor: shape=(), dtype=float32, numpy=26048.0>

In [86]:
from mlsql.utils.conjugate_gradient import conjugate_gradient
from mlsql.utils.ihvp import CGOperator
egrads = manager.egrads(range=[100, 110])
egrads = [tf.reduce_sum(i, axis=0) for i in egrads] #
flattened = [tf.reshape(tensor, [-1]) for tensor in egrads]
packed_vector = tf.concat(flattened, 0)
N = tf.size(packed_vector)
out = conjugate_gradient(CGOperator(N, manager.loss, manager.variables), packed_vector).x
print(out)
# print(manager.iHvp(egrads))

tf.Tensor(
[  32.799644   -32.80015    -34.958447    34.958393    31.079596
  -31.079607     2.4708002   -2.4708238   14.44459    -14.444588
  -12.710449    12.710412     4.9944925   -4.9946246   -9.790886
    9.790905    26.656906   -26.657082   -15.965891    15.96588
  -25.641005    25.641022   -38.865482    38.8655     -25.19015
   25.190117  -121.68458    121.68458     10.141795   -10.140565
   29.2534     -29.253458    66.473915   -66.47376      7.256938
   -7.2568536], shape=(36,), dtype=float32)


In [200]:
trainingKs

[0]

In [20]:
print(fixer.deletions[:100])

tf.Tensor(
[ 6728 11710 16462 21527 18086 18095 16647 18738 23804 23719  2630  5577
  8529  9952 15638 16077 16933 21124 21174 23790 24829 25034  8786 13305
  1876  4003  6707  8457 17122   960  6950 21597  8324 16996 19148 24644
  5404 15300 23185  6035 13859  8447  3925  5810  7640 12830 14761 18936
 22373 22686 26000  7868 10145 19031  9109  9668 19953   293 22322 12811
 16333 18375 18658 24089   707   783  9151  9240 11393  3978 21700  1460
   646  5064  7704 20274 15877    65  9576 16152 19209 20872  1375 21140
  2985 10401 13567 11222 19527  9373 20929  5339 21485  4910 15392 25187
  1435 11530 10991 24478], shape=(100,), dtype=int32)


In [156]:
df_iupi["Method"] = "IUpI (APDS)"

In [206]:
# pdf = pd.concat([df_iupi, df_once, df_oncei, df_rain, df_iup])
pdf = pd.concat([df_iupi_s10, df_iupi_s10_re, df_iupi_s10_init, df_iupi_s10_init_re, df_rain, df_iupi, df_once])
selector = alt.selection_multi()
alt.Chart(pdf, title="Complain of query data vs. K, AC=0").mark_line().encode(
    x = "K",
    y = "Complain",
    color = "Method"
)
# .add_selection(
#     selector
# ).interactive()

alt.Chart(...)

In [207]:
alt.Chart(pdf, title="F1 score of test data vs. K",).mark_line().encode(
    alt.Y('F1',
        scale=alt.Scale(domain=(0.75, 0.85))
    ),
    x = "K",
#     y = "F1",
    color = "Method"
)
# .add_selection(
#     selector
# ).interactive()

alt.Chart(...)

In [23]:
df_time = pd.DataFrame({
    "Methods": ["Once", "Rain", "IUpI (APDS)", "IUpI_Step10"],
    "Model update": [model_time_once, model_time_rain, model_time_iupi, model_time_iupi_s10],
    "Rank computing": [rank_time_once, rank_time_rain, rank_time_iupi, rank_time_iupi_s10],
})

In [24]:
df_time

,Methods,Model update,Rank computing
0,Once,5.325090,5.198538
1,Rain,49.522155,46.045036
2,IUpI (APDS),499.727001,464.421158
3,IUpI_Step10,62.170535,46.446858


In [ ]:
train_diff = np.array(train_diff)
influence_diff = np.array(influence_diff)
print("Training time for once: {:.3f}\nTraining time in total: {:.3f}".format(np.mean(train_diff), np.sum(train_diff)))
print("Influence calculation time for once: {:.3f}\nInfluence calculation time in total: {:.3f}".format(np.mean(influence_diff), np.sum(influence_diff)))
      

In [ ]:
len(rank_list)

In [ ]:
# len(rank_list)
rank_numpy = []
valid_data = np.where(rank_list[-1]!=None)
# print(valid_data)
for rank in rank_list:
    rank_numpy.append(rank[valid_data])
rank_numpy = np.array(rank_numpy)
x = np.arange(0, int(K))
avg = rank_numpy.mean(axis=1)
avg_dis = []
for i in range(len(rank_numpy)-1):
    avg_dis.append(((rank_numpy[i+1]-rank_numpy[i])).mean())

plt.title("Influence of remaining data over the training process")
plt.xlabel("K")
plt.ylabel("Value")
plt.plot(x, avg, 'r-')
# plt.plot((x+0.5)[:-1], avg_dis, 'b-')
plt.legend(["Average on all influences"])#, "Average on diff of influences"])
plt.show()

plt.title("Diff of influence of remaining data over the training process")
plt.xlabel("K")
plt.ylabel("Value")
# plt.plot(x, avg, 'r-')
plt.plot((x+0.5)[:-1], avg_dis, 'b-')
plt.legend(["Average on diff of influences"])
plt.show()

# not a big difference over 20000+ data?
print("First:", rank_numpy[0])
print("Last:", rank_numpy[-1])
print("L2-dis:", ((rank_numpy[0]-rank_numpy[-1])**2).sum()**(1/2))

In [ ]:
rank_numpy.shape
res=np.var(rank_numpy, axis=0)

In [ ]:
res.max()
res.min()
rank_numpy.shape

In [ ]:
import numpy
plt.plot(np.arange(K), AQs, 'r')
plt.plot(np.arange(K), AQs_exact, 'b')
plt.xlabel("K")
plt.ylabel("Complaint")
plt.legend(["Relaxed", "Exact"])
# plt.title("Margin of male average and female average")
plt.show()

In [ ]:
# ACs
# manager.fit()
with tf.GradientTape() as tape:
    male_income = manager.predict_proba(proc.X_test_male)
    female_income = manager.predict_proba(proc.X_test_female)
    Q_male = tf.reduce_mean(male_income[:, 1])
    Q_female = tf.reduce_mean(female_income[:, 1])
    margin = Q_male - Q_female
    AC = 0.0
    AC = tf.stack([AC])
    AQ = tf.stack([margin])
    query_loss = 0
    if AC is not None:
        query_loss += tf.norm(AC - AQ, 1)
# print(AC, AQ)
# print(query_loss)
qgrads = tape.gradient(query_loss, manager.variables)
print(qgrads)
# tf.print(manager.iHvp(qgrads))
with tf.GradientTape() as tape:
    male_income = tf.one_hot(manager.predict(proc.X_test_male), 2)
    female_income = tf.one_hot(manager.predict(proc.X_test_female), 2)
    Q_male = tf.reduce_mean(male_income[:, 1])
    Q_female = tf.reduce_mean(female_income[:, 1])
    margin = Q_male - Q_female
    AC = 0.0
    AC = tf.stack([AC])
    AQ = tf.stack([margin])
    query_loss = 0
    if AC is not None:
        query_loss += tf.norm(AC - AQ, 1)
qgrads = tape.gradient(query_loss, manager.variables)
print(qgrads)
# print(AQ)
# print(query_loss)
# tf.GradientTape().gradient(query_loss, manager.variables)
# len(proc.X_test_male)
# male_income=manager.predict_proba(proc.X_test_male)
# print(tf.reduce_mean(male_income[:, 1]))
# print(tf.reduce_mean(male_income[:, 0]))
# female_income=manager.predict_proba(proc.X_test_female)
# print(tf.reduce_mean(female_income[:, 1]))
# print(tf.reduce_mean(female_income[:, 0]))


In [ ]:
# res_relaxed = AQs
res_exact = AQs
# len(proc.X_test_female)

In [ ]:
# res_relaxed
res_exact

In [ ]:
AQs

In [ ]:
import numpy
plt.plot(np.arange(K), AQs, 'r')
plt.plot(np.arange(K), AQs_exact, 'b')
plt.xlabel("K")
plt.ylabel("Complaint")
plt.legend(["Relaxed", "Exact"])
# plt.title("Margin of male average and female average")
plt.show()

numpy.savetxt("AQs_2000.csv", AQs, delimiter=",",fmt='%1.8f')

numpy.savetxt("AQs_exact_2000.csv", AQs_exact, delimiter=",",fmt='%1.8f')

In [ ]:
# idxs_list = idx_list
# print(idxs_list)
# for i in idx_list:
#     manager.delta.scatter_nd_update([[i]], [0], name="SetDelta")

In [ ]:
import csv
idx = csv.reader(open("idx_2000.csv"))
# index = []
Xremoved = []
Yremoved = []
for i, d in enumerate(idx):
    strs = [str(n) for n in d]
    a_string = "".join(strs)
    index = int(a_string)
    Xremoved.append(proc.Xtrain.numpy()[index])
    Yremoved.append(np.argmax(proc.ytrain.numpy()[index]))
from sklearn import tree
clf = tree.DecisionTreeClassifier()
clf = tree.plot_tree(clf.fit(Xremoved, Yremoved))



In [ ]:
# manager_removed = ModelManager(tf.identity(Xremoved), tf.one_hot(Yremoved, 2), LogReg(proc, 2))
# manager_removed.fit()
# print("LogReg")
# print("On Training Clean\n", classification_report(tf.argmax(tf.one_hot(Yremoved, 2), axis=1).numpy(), manager_removed.predict(tf.identity(Xremoved)).numpy()))
# print("On Testing\n", classification_report(tf.argmax(proc.y_test_full, axis=1).numpy(), manager_removed.predict(proc.X_test_full).numpy()))

In [ ]:
import csv
train_x = csv.reader(open("../processors/adultNoCorr/contradictory_data_X.csv"))

train_data = set()
train_list = []
for d in train_x:
    train_list.append(tuple([int(float(k)) for k in d]))
    train_data.add(train_list[-1])
    
train_idx_group = {
    d:{
        "data": [],
        "target": [],
    } for d in train_data
}

train_y = csv.reader(open("../processors/adultNoCorr/contradictory_data_Y.csv"))
train_list_y = []
for d in train_y:
    train_list_y.append(int(float(d[0])))

print(len(train_list_y))  
print(len(train_list))

import numpy as np
count =0
for i, d in enumerate(train_list):
    train_idx_group[d]['data'].append(i)
    train_idx_group[d]['target'].append(train_list_y[i])
for _, data_target in train_idx_group.items():
    data_target['ones'] = np.sum(np.array(data_target['target'])==1)
    data_target['zeros'] = np.sum(np.array(data_target['target'])==0)
    data_target['ratio'] = data_target['ones'] / len(data_target['target'])
    if data_target['ratio'] >0.4 and data_target['ratio'] <0.6:
        count = count+1

# test_x = csv.reader(open("test_data_X.csv"))

# test_list = []
# for d in test_x:
#     test_list.append(tuple([int(float(k)) for k in d]))
    
# test_y = csv.reader(open("test_data_Y.csv"))
# test_list_y = []
# for d in test_y:
#     test_list_y.append(int(float(d[0])))
    
# print(len(test_list_y))  
# print(len(test_list))

In [ ]:
Xremoved_list = [tuple([int(k) for k in i]) for i in Xremoved]
Yremoved_list = [i for i in Yremoved]
res = []
for d in zip(Xremoved_list, Yremoved_list):
    res.append(d)
# print(sorted(res, key=lambda x:x[0]))
# print(train_idx_group[res[0][0]])
# print(train_idx_group.keys())
ratio_list = []
for r in res:
    if r[0] in train_idx_group:
        d = train_idx_group[r[0]]
        if r[1] == 0:
            ratio_list.append(d['zeros']/len(d['data']))
        else:
            ratio_list.append(d['ones']/len(d['data']))
    else:
        ratio_list.append(-1)
print((np.array(ratio_list)==-1).sum()/len(ratio_list))
# print(ratio_list)
granu = 0.01
x = np.arange(0, 1, granu)
n = len(x)
y = np.zeros(n)
y_zeros = np.zeros(n)
# y_ones = np.zeros(n)
for i, r in enumerate(ratio_list):
    if r >=0:
        y[int((r) / granu)] += 1
# print(len(ratio_list), len(train_idx_group))
for i, r in train_idx_group.items():
    y_zeros[int(r['zeros']/len(r['data']) / granu)] += r['zeros']
    y_zeros[int(r['ones']/len(r['data']) / granu)] += r['ones']
#     y_ones[int(r['ones']/len(r['data']) / granu)] += 1
x += (granu / 2)
plt.plot(x, y, 'r')
plt.plot(x, y_zeros, 'b')
# plt.plot(x, y_ones, 'g')
plt.show()

In [ ]:
# import csv
# idx = csv.reader(open("idx_2000.csv"))

# Xremoved_list = [tuple(i) for i in Xremoved]
# Xremained = [] 
# for i, train_data in enumerate(proc.Xtrain.numpy().tolist()):
#     if tuple(train_data) not in Xremoved_list:
#         Xremained.append(train_data)
        
        


In [ ]:
x = manager.delta.numpy()
idx_list = []
for idx, n in enumerate(x):
    if n == 0:
        idx_list.append(idx)
print(len(idx_list))
# print(x.sum())
# rank = ranker.predict()
for i in idx_list:
    manager.delta.scatter_nd_update([[i]], [1], name="SetDelta")
print(manager.delta.numpy().sum())
rank = ranker.predict()
for i in idx_list:
    manager.delta.scatter_nd_update([[i]], [0], name="SetDelta")
# manager.delta[idx_list] = 0
# print(x.sum())
rank = rank.numpy()
print(rank.min(), rank.max(), len(rank))
print(rank)

In [ ]:
np.savetxt("idx_2000.csv", idx_list, delimiter=",",fmt='%d')

In [ ]:
from scipy.optimize import curve_fit
from numpy import exp

granu = 0.1
x = np.arange(-3.6, 3.5, granu)
n = len(x)
y = np.zeros(n)
y_remove = np.zeros(n)
for i, r in enumerate(rank):
    if i in idx_list:
        y_remove[int((r + 3.6) / granu)] += 1
    else:
        y[int((r + 3.6) / granu)] += 1
x += (granu / 2)
mean = sum(x * y) / sum(y)
sigma = np.sqrt(sum(y * (x - mean)**2) / sum(y))
print(mean, sigma)

def gaus(x,a,x0,sigma):
    return a*exp(-(x-x0)**2/(2*sigma**2))

popt, pcov = curve_fit(gaus, x, y, p0=[max(y), mean, sigma])
# plt.plot(x, y, 'r')
plt.bar(x, y, color = 'r', width = granu, alpha=0.7)
# plt.plot(x, y_remove, 'g')
plt.bar(x, y_remove, color = 'g', width = granu, alpha=0.7)
# plt.plot(x, gaus(x, *popt), 'b')
plt.show()

In [ ]:
idxs = influence.argsort()
influence_sorted = np.sort(influence)
mask = (influence_sorted < 0)
influence_neg = influence_sorted[mask]
influence_pos = influence_sorted[(1-mask).astype(np.bool)]
idxs_neg = idxs[mask].tolist()
idxs_pos = idxs[(1-mask).astype(np.bool)].tolist()
print("Number neg", len(idxs_neg))
print("Number pos", len(idxs_pos))


In [ ]:
import csv
with open("../processors/adultNoCorr/contradictory_data_X.csv", 'r') as f:
    X_data = csv.reader(f)
    X_process_data = []
    for d in X_data:
        X_process_data.append(tuple([int(float(num)) for num in d]))
    X_data = set(X_process_data)
all_train = proc.Xtrain.numpy()
neg_contradict = 0
pos_contradict = 0
neg_dict = {}
pos_dict = {}
for i, id in enumerate(idxs_neg):
    if tuple(all_train[id]) in X_data:
        neg_contradict += 1
    neg_dict[i-len(idxs_neg)] = neg_contradict / (i + 1)
for i, id in enumerate(idxs_pos):
    if tuple(all_train[id]) in X_data:
        pos_contradict += 1
    pos_dict[i] = (neg_contradict + pos_contradict) / (len(idxs_neg) + i + 1)
# amongst all negative and positive, how many in contradict
neg_cont_ratio = neg_contradict / len(idxs_neg)
pos_cont_ratio = pos_contradict / len(idxs_pos)
print(neg_cont_ratio)
print(pos_cont_ratio)

# all_possible_contradict = 0
# for d in X_data:
#     if tuple(d) in all_train:
#         all_possible_contradict += 1
# amongst all X_data in training, how much find out
# recall_ratio = contradict / all_possible_contradict
# print(recall_ratio)

In [ ]:
plt.plot(np.arange(K), res_relaxed, 'r')
plt.plot(np.arange(K), res_exact, 'b')
plt.xlabel("K")
plt.ylabel("AQ")
plt.legend(["Relaxed", "Exact"])
# plt.title("Margin of male average and female average")
plt.show()

In [ ]:
# sess = tf.InteractiveSession()
tf.print(PC)
tf.print(PQ)
query_loss = tf.nn.softmax_cross_entropy_with_logits(labels=PQ, logits=PC)
query_loss
# qgrads = tf.GradientTape().gradient(query_loss, manager.variables)
# qgrads
# tf.nn.softmax_cross_entropy_with_logits(PC, PQ)
# print(proc.complaint_mask)
# print(proc.y_test_full[proc.complaint_mask])
# print(PC.shape)
# print(PQ.shape)

In [ ]:
# plt.figure(figsize=(16,8))

# # fixer = next(iter(exp_results.values()))[2]
# # ntruth = fixer.truth.numpy().sum()
# # plt.plot(range(ntruth), np.linspace(0, 1, ntruth), label="Ground Truth")

# # rand_prob = ntruth / len(fixer.truth)
# # k = np.arange(len(fixer.get_deletions()))
# # plt.plot(k, k * rand_prob / ntruth, label = "Random")


# for key, (manager, ranker, _, _, _, _) in exp_results.items():

# #     recalls = np.empty(len(fixer.get_deletions()))
#     delta = np.ones(manager.X.shape[0])


# #     k = np.arange(len(fixer.get_deletions()))

#     for i in k:
#         delta[fixer.get_deletions()[i]] = 0
#         recalls[i] = recall_score(corrsel, 1 - delta)

#     plt.plot(k+1, recalls, label = key)
    
# plt.xlabel("K")
# plt.ylabel("Value")
# plt.title(f"Influence with retrain, vary K, corrupt = {corruption_rate}")
# plt.legend()

In [ ]:
PQ[:,0]
# plt.plot(PQ[:,0], label="Relaxed")

In [ ]:
from sklearn.metrics import accuracy_score

y_predict = tf.one_hot(manager.predict(proc.X_test_full), 2)

print(y_predict)
print(proc.y_test_full.shape)
target_names = ['class 0', 'class 1']
print(classification_report(proc.y_test_full, y_predict, target_names=target_names))

In [ ]:
from io import BytesIO

In [ ]:
f = BytesIO()

In [ ]:
a = np.asarray([], dtype=np.float32)

In [ ]:
np.savez_compressed(f, a)

In [ ]:
len(f.getvalue())

In [ ]:
f.getvalue()

In [ ]:
import compress_pickle

In [ ]:
compress_pickle.dump